In [5]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import gym
from collections import deque
import random
import time
import yaml
from klqr import *
from train_utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
with open('config.yml','r') as ymlfile:
    config = yaml.load(ymlfile)
    
tf.reset_default_graph()
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))

In [7]:
env = gym.make('LQ-v0')
agent = klqr(config,sess)
agent.build_model()
train_agent(agent,env,max_timesteps=40000, n_transitions_between_updates=100,
                n_optim_steps_per_update=200,
                n_iters_per_p_update=5,
                )

z shape: (?, 3)
********** Iteration 1 ************
	Episode Len: 100
	Last Episode Cost: 20576.800923 vs optimal 0.000000
********** Iteration 2 ************
	Episode Len: 100
	Last Episode Cost: 248185.250880 vs optimal 9.617052
********** Iteration 3 ************
	Episode Len: 100
	Last Episode Cost: 191801.578023 vs optimal 10.486355
********** Iteration 4 ************
	Episode Len: 100
	Last Episode Cost: 96586.762941 vs optimal 16.458772
********** Iteration 5 ************
[[ 1.0061483  -0.01611208 -0.01615164]
 [-0.01611208  1.0599922   0.07317833]
 [-0.01615164  0.07317833  1.1007798 ]]
	Episode Len: 100
	Last Episode Cost: 197892.401228 vs optimal 10.831921
********** Iteration 6 ************
	Episode Len: 100
	Last Episode Cost: 586981.453949 vs optimal 7.788326
********** Iteration 7 ************
	Episode Len: 100
	Last Episode Cost: 399779.084747 vs optimal 24.924905
********** Iteration 8 ************
	Episode Len: 100
	Last Episode Cost: 104649.099523 vs optimal 3.904683


KeyboardInterrupt: 

In [4]:
#test

agent.max_a = 100.0
agent.min_a = -100.0
train_agent(agent,env,max_timesteps=1000,training=False)

********** Iteration 1 ************
	Episode Len: 100
	Last Episode Cost: 6737.330726 vs optimal 0.000000
********** Iteration 2 ************
	Episode Len: 100
	Last Episode Cost: 6858.035857 vs optimal 6.501063
********** Iteration 3 ************
	Episode Len: 100
	Last Episode Cost: 6897.710797 vs optimal 33.096960
********** Iteration 4 ************
	Episode Len: 100
	Last Episode Cost: 6676.397342 vs optimal 1.639398
********** Iteration 5 ************
	Episode Len: 100
	Last Episode Cost: 6928.350092 vs optimal 5.943126
********** Iteration 6 ************
	Episode Len: 100
	Last Episode Cost: 6925.595812 vs optimal 13.776395
********** Iteration 7 ************
	Episode Len: 100
	Last Episode Cost: 6928.536660 vs optimal 12.420773
********** Iteration 8 ************
	Episode Len: 100
	Last Episode Cost: 6848.767705 vs optimal 10.985332
********** Iteration 9 ************
	Episode Len: 100
	Last Episode Cost: 6926.063003 vs optimal 18.695012
********** Iteration 10 ************
	Epi

In [56]:
#test

agent.max_a = 100.0
agent.min_a = -100.0
train_agent(agent,env,max_timesteps=1000,training=False)

********** Iteration 1 ************
	Episode Len: 100
	Last Episode Cost: 915.910508 vs optimal 0.000000
********** Iteration 2 ************
	Episode Len: 100
	Last Episode Cost: 922.012108 vs optimal 16.436779
********** Iteration 3 ************
	Episode Len: 100
	Last Episode Cost: 1117.233503 vs optimal 61.685952
********** Iteration 4 ************
	Episode Len: 100
	Last Episode Cost: 948.558038 vs optimal 14.255025
********** Iteration 5 ************
	Episode Len: 100
	Last Episode Cost: 913.919872 vs optimal 101.965660
********** Iteration 6 ************
	Episode Len: 100
	Last Episode Cost: 1090.863860 vs optimal 219.421257
********** Iteration 7 ************
	Episode Len: 100
	Last Episode Cost: 1064.240353 vs optimal 225.311849
********** Iteration 8 ************
	Episode Len: 100
	Last Episode Cost: 1262.386139 vs optimal 39.287106
********** Iteration 9 ************
	Episode Len: 100
	Last Episode Cost: 938.289780 vs optimal 3.604805
********** Iteration 10 ************
	Epi

In [85]:
sess.run(agent.A)

array([[nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan]], dtype=float32)

In [84]:
sess.run(agent.B)

array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan]], dtype=float32)

In [83]:
sess.run(agent.Q)

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

In [82]:
sess.run(agent.R)

array([[nan]], dtype=float32)

In [40]:
A,B,Q,R = (env.env.A, env.env.B, env.env.Q, env.env.R)
P = Q
for k in range(100):
    P = Q + A.T @ P @ A - A.T @ P.T @ B @ np.linalg.inv(R + B.T @ P @ B ) @ B.T @ P @ A
    P = 0.5*(P + P.T)

In [41]:
P

array([[2.23999771, 0.54694411, 0.19286546],
       [0.54694411, 2.35082156, 0.54694411],
       [0.19286546, 0.54694411, 2.23999771]])

In [42]:
sess.run(agent.P)

array([[1.7117685 , 0.55636847, 0.07273098],
       [0.55636847, 1.7515693 , 0.6495986 ],
       [0.07273098, 0.6495986 , 1.8076872 ]], dtype=float32)